In [4]:
from tkinter import *
window=Tk()
import re
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords ,wordnet
from nltk.stem import PorterStemmer
import networkx as nx
import random

In [5]:
allurls=[]
search_index=[]
bool_index=[]
inverted_index={}

In [6]:
def build_bool_index(word):
    docs=allurls
    bool_index = np.zeros(len(docs), dtype=int)
    
    for j in range(len(docs)):
        
        if word in get_terms(docs[j]):
            bool_index[j] = 1
    return bool_index


In [7]:
class PageRank():
    def __init__(self, node_count, graph, iters, dr):
        self.node_count = node_count
        self.graph = graph
        self.page_rank = np.zeros(node_count)
        self.itres = iters
        self.dr = dr
        
    def calc_outgoing_links(self, node):
        outgoing_links = 0
        for i in range(self.node_count):
            if self.graph[i] == 1:
                outgoing_links += 1
        return outgoing_links
    
    def calc_pr_for_single_node(self,node):
        page_rank = 0
        for o in range(self.node_count):
            if self.graph[o] == 1:
                page_rank += (1 - self.dr) + self.dr * self.page_rank[o] / self.calc_outgoing_links(o)
        return page_rank
    
    def calc_full_page_rank(self):
        init_page_rank = 1 / self.node_count
        for i in range(self.node_count):
            self.page_rank[i] = init_page_rank  
        for j in range(self.itres):
            temp_page_rank = np.zeros(self.node_count)
            for i in range(self.node_count):
                temp_page_rank[i] = self.calc_pr_for_single_node(i)
            lb.insert(END,f"Page Rank after {j} iteration:")
            for i in range(self.node_count):
                lb.insert(END,f"Page rank for node {i} is {temp_page_rank[i]}")
            lb.insert(END,"---------------------------")
            self.page_rank = temp_page_rank
        

In [8]:
def pg_search():
    tokens1 = word_tokenize(tSh.get())
    for i in tokens1:
        pr = PageRank(5, build_bool_index(i), 20, 0.85)
        pr.calc_full_page_rank()

In [9]:
def build_inverted_index(docs):
    
    for i in range (10):
        terms= get_terms(docs[i])
        for term in terms:
            if term in inverted_index.keys():
                if i not in inverted_index[term]:
                    inverted_index[term].append(i)
            else:
                inverted_index.update({term:[i]})
    return inverted_index

In [10]:
def get_terms(doc):
        page_content = requests.get(doc).text
        soup = BeautifulSoup(page_content, 'html.parser')
        results = soup.get_text()
        stopset = set(stopwords.words('english'))
        stemmer = PorterStemmer()
        tokens = wordpunct_tokenize(results)
        clean = [token.lower() for token in tokens if token.lower() not in stopset ]
        final = [stemmer.stem(word) for word in clean]
        return set(final)

In [11]:
def inverted_search(query):
    query=query.lower()
    stemmer = PorterStemmer()
    query=stemmer.stem(query)
    for i in inverted_index[query]:
        search_index=allurls[i] 
        
        
   
    return inverted_index[query]


In [12]:
class crawler:
    
    def __init__(self):
        self.to_visit = list()
        self.visited = set()

    def fetch(self, url):
        print('now fetching.. ' , url)
        
        try:
            res = requests.get(url).content

            return res
        except:
            return

    def get_current_url(self):
        res = self.to_visit.pop()

        while res in self.visited:
            print('already visited',res)
            res = self.to_visit.pop()

        return res

    def get_links(self , content ):
        urls = re.findall('<a href="([^"]+)">', str(content))
        print('urls are', urls)
        for url in urls :
            if url != '#':
                allurls.append(url)
                pattern = re.compile('http?')

                if pattern.match(url):
                    self.to_visit.append(url)

    def  crawl(self, url, depth= 5):
        self.to_visit.append(url)
        while len(self.visited) <= depth:
            
            
            current_url = self.get_current_url()
            

            content = self.fetch(current_url)

            self.visited.add(current_url)
            self.get_links(content)

        print('not visted',self.to_visit)
        print('visited',self.visited)


In [13]:
def start() :
    inverted_index=build_inverted_index(allurls)
   


In [14]:
def compute_tf(w,doc):
    page_content = requests.get(doc).text
    soup = BeautifulSoup(page_content, 'html.parser')
    results = soup.get_text()
    words = word_tokenize(results)
    freq = [1  for s in words if s == w]
    
    return len(freq)/ len(words)


def compute_idf(w,search_index):
    sents=len(allurls)
    return np.log(sents/ len(search_index))


def compute_tfidf(word,search_index):
    for i in search_index:
        tf=compute_tf(word,allurls[i])
        
    return compute_idf(word,search_index)*tf

In [15]:
def df_search():
    tokens1 = word_tokenize(tSh.get())
    for i in tokens1:
       
        search_index=inverted_search(i)
        tf=compute_tfidf(i,search_index)
        for j in search_index:
            
            lb.insert(END,{tf,i,allurls[j]})


In [16]:


# Add directed edges in graph
def add_edges(g, pr):
	for each in g.nodes():
		for each1 in g.nodes():
			if (each != each1):
				ra = random.random()
				if (ra < pr):
					g.add_edge(each, each1)
				else:
					continue
	return g

# Sort the nodes
def nodes_sorted(g, points):
	t = np.array(points)
	t = np.argsort(-t)
	return t

# Distribute points randomly in a graph
def random_Walk(g):
	rwp = [0 for i in range(g.number_of_nodes())]
	nodes = list(g.nodes())
	r = random.choice(nodes)
	rwp[r] += 1
	neigh = list(g.out_edges(r))
	z = 0
	
	while (z != 10000):
		if (len(neigh) == 0):
			focus = random.choice(nodes)
		else:
			r1 = random.choice(neigh)
			focus = r1[1]
		rwp[focus] += 1
		neigh = list(g.out_edges(focus))
		z += 1
	return rwp




In [32]:
def thrd_search():
    tokens1 = word_tokenize(tSh.get())
    for i in tokens1:       
        search_index=inverted_search(i)
        # Main
    # 1. Create a directed graph with N nodes
        g = nx.DiGraph()

        N = len(search_index)
        g.add_nodes_from(range(N))

        # 2. Add directed edges in graph
        g = add_edges(g, 0.4)

        # 3. perform a random walk
        points = random_Walk(g)

        # 4. Get nodes rank according to their random walk points
        sorted_by_points = nodes_sorted(g, points)
        lb.insert(END,"PageRank using Random Walk Method")
        for i in sorted_by_points:
            lb.insert(END,allurls[i])

In [34]:
from tkinter import *
window=Tk()
# add widgets here
crawlbtn=Button(window, text="crawler",command = lambda:[crawler().crawl(tURl.get()),start()])
tfIdfbtn=Button(window, text="tf idf",command = lambda:[df_search(), lb.delete(0,END)])
pgrbtn=Button(window, text="page rank",command = lambda:[pg_search(), lb.delete(0,END)])
thrdbtn=Button(window, text="3rd method",command = lambda:[thrd_search(), lb.delete(0,END)])

crawlbtn.place(x=90, y=200)
tfIdfbtn.place(x=60, y=250)
pgrbtn.place(x=120, y=250)
thrdbtn.place(x=80, y=300)


lblurl=Label(window, text="enter your URL ..", fg='red', font=("Helvetica", 16))
lblurl.place(x=50, y=50)
tURl=Entry(window, text="")

tURl.place(x=55, y=80)

lblsh=Label(window, text="enter your search ..", fg='red', font=("Helvetica", 16))
lblsh.place(x=50, y=110)

tSh=Entry(window, text="")
tSh.place(x=55, y=150)
lb=Text(window,width=110, height=40)
lb.place(x=300, y=50)

window.title('Hello Python')
window.geometry("1200x700+10+20")
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-34-d8dc026f0622>", line 7, in <lambda>
    thrdbtn=Button(window, text="3rd method",command = lambda:[thrd_search(), lb.delete(0,END)])
  File "C:\Users\User\anaconda3\lib\tkinter\__init__.py", line 3602, in delete
    self.tk.call(self._w, 'delete', index1, index2)
_tkinter.TclError: bad text index "0"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-34-d8dc026f0622>", line 7, in <lambda>
    thrdbtn=Button(window, text="3rd method",command = lambda:[thrd_search(), lb.delete(0,END)])
  File "C:\Users\User\anaconda3\lib\tkinter\__init__.py", line 3602, in delete
    self.tk.call(self._w, 'delete', index1, index2)
_tkinter.TclError: bad text index 